In [1]:
from tensorflow import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.4.0
2.4.0


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
SAMPLE = 1000000
EPOCHS = 15

# Florida voter
df = pd.read_csv('../dataverse_files/fl_reg_name_race.csv.gz')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
sdf = df[df.race.isin(['multi_racial', 'native_indian', 'other', 'unknown']) == False].sample(SAMPLE, random_state=21)
del df

# Additional features
sdf['name_last'] = sdf.name_last.str.title()

sdf

,name_first,name_last,race
841323,Torres,Jose,hispanc
1408926,Da Silva,Amanda,nh_white
1733118,Mc Ghee,Sandra,nh_white
13104513,KARAM,Melinda,nh_white
9156114,BREWER,Laia,nh_black
...,...,...,...
3076722,Antunez Avila,Robert,hispanc
10023679,DAVIS,Wyatt,nh_white
5846252,Scott,Jacquelyn,nh_white
5959131,Parton,Douglas,nh_white


In [3]:
rdf = sdf.groupby('race').agg({'name_last': 'count'})
rdf.to_csv('../dataverse_files/fl_voter_reg/lstm/fl_ln_race.csv', columns=[])
rdf

,name_last
race,
asian,19431
hispanc,166865
nh_black,142675
nh_white,671029


In [4]:
sdf.groupby('race').agg({'name_last': 'nunique'})

,name_last
race,
asian,8933
hispanc,17512
nh_black,33422
nh_white,25575


## Preprocessing the input data

In [5]:
# last name only
sdf['name_last_name_first'] = sdf['name_last']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1103
Max feature len = 25, Avg. feature len = 5


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [6]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.models import load_model

max_features = num_words # 20000
feature_len = 20 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 20)
X_test shape: (200000, 20)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 4)
y_test shape: (200000, 4)


## Save model

In [11]:
model.save('../dataverse_files/fl_voter_reg/lstm/fl_all_ln_lstm.h5')

In [12]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('../dataverse_files/fl_voter_reg/lstm/fl_all_ln_vocab.csv', index=False, encoding='utf-8')

# Uncertainity Model

In [25]:
print('Build uncertainty model...')

#model_uncrtn = Sequential()
input_ = keras.layers.Input(shape=(feature_len))
layer_ = Embedding(num_words, 32) (input_)
layer_ = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(layer_, training=True)
#layer_ = Dropout(0.25)(layer_, training=True)
output = Dense(num_classes, activation='softmax') (layer_)

model_uncrtn = keras.models.Model(inputs=input_, outputs=output)

# try using different optimizers and different optimizer configs
model_uncrtn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model_uncrtn.summary())

Build uncertainty model...
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 20)]              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 20, 32)            35296     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 516       
Total params: 118,244
Trainable params: 118,244
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
print('Train...')
model_uncrtn.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model_uncrtn.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/15
22500/22500 [==============================] - 422s 19ms/step - loss: 0.7446 - accuracy: 0.7364 - val_loss: 0.6784 - val_accuracy: 0.7628
Epoch 2/15
22500/22500 [==============================] - 413s 18ms/step - loss: 0.6694 - accuracy: 0.7670 - val_loss: 0.6567 - val_accuracy: 0.7720
Epoch 3/15
22500/22500 [==============================] - 405s 18ms/step - loss: 0.6506 - accuracy: 0.7745 - val_loss: 0.6491 - val_accuracy: 0.7761
Epoch 4/15
22500/22500 [==============================] - 397s 18ms/step - loss: 0.6413 - accuracy: 0.7775 - val_loss: 0.6452 - val_accuracy: 0.7778
Epoch 5/15
22500/22500 [==============================] - 397s 18ms/step - loss: 0.6338 - accuracy: 0.7806 - val_loss: 0.6428 - val_accuracy: 0.7779
Epoch 6/15
22500/22500 [==============================] - 400s 18ms/step - loss: 0.6328 - accuracy: 0.7816 - val_loss: 0.6396 - val_accuracy: 0.7792
Epoch 7/15
22500/22500 [==============================] - 408s 18ms/step - loss: 0.6309 - accurac

In [27]:
predictions = []

ITER=100

for _ in range(ITER):
    predictions.append(model_uncrtn.predict(X_test, verbose=1))

6250/6250 [==============================] - 29s 5ms/step


In [28]:
predict_array = np.array(predictions)

In [29]:
# Creating a seperate array for each measurement
mean_arr = predict_array.mean(axis=0).reshape(-1,4)
std_arr = predict_array.std(axis=0).reshape(-1,4)
pct_5_arr = np.quantile(predict_array, .05, axis=0).reshape(-1,4)
pct_95_arr = np.quantile(predict_array, .95, axis=0).reshape(-1,4)

In [30]:
# Selecting the class based on teh mean value with the highest probability
final_pred_arr = mean_arr.argmax(axis=1)

In [31]:
final_pred_arr.shape

(200000,)

# Creating Final DataFrame

In [34]:
target_names = list(sdf.race.astype('category').cat.categories)

In [42]:
predict_df = pd.DataFrame(columns = ['pred', 'category' , 'pred_5', 'pred_95', 'pred_se'])

for i in range(len(final_pred_arr)):
    pred = final_pred_arr[i]
    category = target_names[pred]
    pred_5 = pct_5_arr[i,pred]
    pred_95 = pct_95_arr[i,pred]
    pred_se = std_arr[i,pred]
    
    predict_df = predict_df.append({'pred':pred, 
                                    'category': category,
                                    'pred_5':pred_5, 
                                    'pred_95': pred_95, 
                                    'pred_se':pred_se}, ignore_index=True)

In [43]:
predict_df

,pred,category,pred_5,pred_95,pred_se
0,3,nh_white,0.466160,0.776006,0.091654
1,3,nh_white,0.784752,0.890695,0.034224
2,3,nh_white,0.324650,0.490653,0.051946
3,3,nh_white,0.746301,0.872112,0.039808
4,3,nh_white,0.286234,0.748420,0.138514
...,...,...,...,...,...
199995,3,nh_white,0.653034,0.790576,0.041229
199996,2,nh_black,0.699155,0.952282,0.081714
199997,3,nh_white,0.652905,0.873807,0.066446
199998,3,nh_white,0.883809,0.931329,0.013905


In [49]:
predict_df['category'].value_counts()

nh_white    159340
hispanc      27666
nh_black     11502
asian         1492
Name: category, dtype: int64